In [1]:
import requests
from bs4 import BeautifulSoup
import time
from openpyxl import load_workbook, Workbook
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Replace "path_to_your_model.h5" with the actual path to your trained model
model_path = "D:/All_Coding_stuff/FinancialDashboard/Model/model.h5"
model = load_model(model_path)

ticker = 'RELIANCE'
url = f'https://www.google.com/finance/quote/{ticker}:NSE?hl=en'
data_file_path = 'D:/All_Coding_stuff/FinancialDashboard/Data.xlsx'
prediction_file_path = 'D:/All_Coding_stuff/FinancialDashboard/Prediction.xlsx'

# Define the number of time steps and features
time_steps = 10  # Predict for the next 3 days
num_features = 1  # Assuming you're using only the Close price as a feature

# Load the data file
data_wb = load_workbook(data_file_path)
data_sheet = data_wb.active

# Create the prediction file
pred_wb = Workbook()
pred_sheet = pred_wb.active
pred_sheet['A1'] = 'Date'
pred_sheet['B1'] = 'Current Price'
for i in range(1, time_steps + 1):
    pred_sheet.cell(row=1, column=2+i).value = f'Prediction {i}'

# Initialize a list to store the latest data points
latest_data = []
# Store the last observed price
last_observed_price = None

# Create MinMaxScaler instance
scaler = MinMaxScaler(feature_range=(0, 1))

while True:
    web = requests.get(url)
    soup = BeautifulSoup(web.text, 'html.parser')
    class1 = 'YMlKec'
    element = soup.find(class_=class1)

    if element:
        price = float(element.text.strip()[1:].replace(",", ""))
        print("Price:", price)

        current_utc_datetime = datetime.utcnow()
        current_date = current_utc_datetime.strftime("%Y-%m-%d")
        print("Date and Time (UTC):", current_date)

        # Only proceed if the current price is different from the last observed price
        if price != last_observed_price:
            last_observed_price = price

            # Append price and date to the data file
            data_sheet.append([current_date, price])
            data_wb.save(filename=data_file_path)
            print("Data appended successfully.")

            # Append the latest price to the list of latest data points
            latest_data.append(price)

            # Keep only the latest 'time_steps' data points
            if len(latest_data) > time_steps:
                latest_data.pop(0)

            # Prepare the input data for prediction
            if len(latest_data) == time_steps:
                # Scale the data
                scaled_data = scaler.fit_transform(np.array(latest_data).reshape(-1, 1))

                # Reshape the data for model input
                X = np.array([scaled_data]).reshape(1, time_steps, 1)

                # Make predictions for the next 3 days
                predicted_prices = []
                for _ in range(time_steps):
                    predicted_price = model.predict(X)
                    predicted_prices.append(predicted_price[0][0])
                    # Update X with the latest prediction and remove the oldest prediction
                    X = np.roll(X, -1)
                    X[0, -1] = predicted_price

                # Inverse transform the predicted prices to get them back to the original scale
                predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1)).flatten()

                # Append the current price and predicted prices to the prediction file
                pred_sheet.append([current_date, price] + list(predicted_prices))
                pred_wb.save(filename=prediction_file_path)
                print("Predicted Prices for the next 3 days:", predicted_prices)
        else:
            print("Skipping prediction since current price is the same as the last observed price.")

    else:
        print("Element not found")
    time.sleep(1)




Price: 2147.0
Date and Time (UTC): 2024-04-21
Data appended successfully.
Price: 2147.0
Date and Time (UTC): 2024-04-21
Skipping prediction since current price is the same as the last observed price.
Price: 2147.0
Date and Time (UTC): 2024-04-21
Skipping prediction since current price is the same as the last observed price.
Price: 2147.0
Date and Time (UTC): 2024-04-21
Skipping prediction since current price is the same as the last observed price.
Price: 2147.0
Date and Time (UTC): 2024-04-21
Skipping prediction since current price is the same as the last observed price.
Price: 2147.0
Date and Time (UTC): 2024-04-21
Skipping prediction since current price is the same as the last observed price.
Price: 2147.0
Date and Time (UTC): 2024-04-21
Skipping prediction since current price is the same as the last observed price.
Price: 2147.0
Date and Time (UTC): 2024-04-21
Skipping prediction since current price is the same as the last observed price.
Price: 2147.0
Date and Time (UTC): 2024-04

KeyboardInterrupt: 